In [ ]:
!pip install --upgrade --quiet  langchain-core langchain-community langchain-openai grandalf

<a href="https://colab.research.google.com/github/qxr777/llm-application-code/blob/master/langchain_expression_language/parallelism-example.ipynb" target="_parent">Open In Colab</a>

In [ ]:
# 三选一 加载OPENAI_API_KEY

# 本地运行，包含.env文件
# from dotenv import load_dotenv
# load_dotenv()

# 本地运行，不包含.env文件
# import os
# os.environ['OPENAI_API_KEY'] = 'sk-123456789'

# google colab运行，需要配置secret key
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel

model = ChatOpenAI()
chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

# 打印chain的计算图
combined.get_graph().print_ascii()

               +--------------------------+                
               | Parallel<joke,poem>Input |                
               +--------------------------+                
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+*                     +------------+     
                   ***               ***                   
                      ***         ***   

In [3]:
%%time
chain1.invoke({"topic": "bears"})

CPU times: user 135 ms, sys: 33.6 ms, total: 169 ms
Wall time: 1.4 s


AIMessage(content="Why did the bear break up with his girlfriend? Because he couldn't bear the relationship anymore!", response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [4]:
%%time
chain2.invoke({"topic": "bears"})

CPU times: user 35.8 ms, sys: 6.77 ms, total: 42.6 ms
Wall time: 1.54 s


AIMessage(content='In the forest they roam, majestic and free\nBears embody strength and wild beauty.', response_metadata={'finish_reason': 'stop', 'logprobs': None})

In [5]:
%%time
combined.invoke({"topic": "bears"})

CPU times: user 78.8 ms, sys: 10.5 ms, total: 89.3 ms
Wall time: 1.98 s


{'joke': AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", response_metadata={'finish_reason': 'stop', 'logprobs': None}),
 'poem': AIMessage(content="In the forest they roam, strong and wild,\nBears embody nature's untamed child.", response_metadata={'finish_reason': 'stop', 'logprobs': None})}